In [2]:
import re
import os
import sqlite3
import time

import pandas as pd

from tqdm import tqdm
from joblib import Parallel, delayed

import matplotlib.pyplot as plt

Используя данные телеметрии, определить топ 10 самых популярных сценариев запуска алгоритмов в рамках одной сессии.  
Визуализировать полученные данные на дэшборде. Инструмент и тип графиков на усмотрение автора.

В ответе необходимо указать:

- Общее количество сессий.
- Количество выполнений каждого сценария из топ 10 списка.
- Последовательность запуска алгоритмов для каждого сценария из топ 10.

Структура данных

[ApplicationStarted] – Начало сессии
[ApplicationClosed] — Окончание сессии
[AlgorithmStarted] — Был запущен алгоритм
[AlgorithmCompletedTime] — Алгоритм завершен
Algorithm {id} — Название алгоритма

In [128]:
def get_algorithm(x):
    if 'result' in x.lower():
        return ' '.join(x.split('.')[0].split(' ')[1:])
    elif 'count' in x.lower():
        return ' '.join(x.split(',')[0].split(' ')[1:])
    else:
        return ' '.join(x.split('|')[0].split(' ')[1:])

In [129]:
def execute_sql(query, conn):
    return pd.read_sql(ex1_sql_query, con=conn)

### EXTRACT & TRANSFORM

In [3]:
log_paths = [path for path in os.listdir() if '.log' in path]
count = 0
limit = len(log_paths)

columns = ('session', 'code', 'model', 'date', 'info', 'log_path')
df = pd.DataFrame(columns=columns)

# error_rows = []
error_paths = []

for log_path in tqdm(log_paths):
    if count == limit:
        break
    with open(log_path) as file:
        log = file.read()
    for row in log.split('\n'):
        if row != '':
            elements = re.sub('[\[\]]', ' ', row).strip().split('  ')
            try:
                df.loc[len(df)] = elements + [log_path]
            except ValueError:
#                 error_rows.append(row)
                error_paths.append(log_path)
    count += 1

100%|██████████████████████████████████████████████████████████████████████████████| 3721/3721 [05:18<00:00, 11.67it/s]


In [107]:
df['algo'] = df['info'].apply(lambda x: get_algorithm(x))
df['message'] = df['info'].apply(lambda x: x.split(' ')[0])
df.head(10)

### LOAD

In [109]:
with sqlite3.connect(":memory:") as conn:
    df[df.message == 'AlgorithmStarted'].to_sql(name="logs", con=conn, index=False)
    
ex1_sql_query = """
                    with foo1 as (
                        with foo as (
                            select log_path, group_concat(algo, ' -> ') over(partition by log_path) pattern
                            from logs)
                        select log_path, pattern
                        from foo
                        group by log_path)
                    select pattern, count(pattern) pattern_counts
                    from foo1
                    group by pattern
                    order by 2 desc;
                """

top10 = execute_sql(ex1_sql_query, conn)
top10.to_csv('top10.csv', index=False)

In [132]:
print(f'Общее количество сессий: {sum(top10.pattern_counts)}')

Общее количество сессий: 3709


In [139]:
print('Топ 10 самых популярных последовательностей алгоритмов: ')
for algo_seq in top10.pattern.iloc[:10].values:
    print(algo_seq)

Топ 10 самых популярных последовательностей алгоритмов: 
Algorithm 7
Algorithm 7 -> Algorithm 10 -> Algorithm 15
Algorithm 17
Algorithm 10
Algorithm 7 -> Algorithm 10
Algorithm 7 -> Algorithm 8 -> Algorithm 10 -> Algorithm 15
Algorithm 13
Algorithm 7 -> Algorithm 7
Algorithm 7 -> Algorithm 8 -> Algorithm 10
Algorithm 6 -> Algorithm 7 -> Algorithm 8 -> Algorithm 10
